<a href="https://colab.research.google.com/github/DayalStrub/ecir2021tutorial/blob/main/other/2_rm3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q python-terrier
!pip install -q git+https://github.com/terrier-org/pyterrier.git

     |████████████████████████████████| 1.1MB 9.4MB/s 
     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 163kB 20.1MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 5.5MB 26.6MB/s 
     |████████████████████████████████| 133kB 47.3MB/s 
     |████████████████████████████████| 1.8MB 46.3MB/s 
     |████████████████████████████████| 645kB 36.3MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 


In [2]:
import pyterrier as pt
pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


terrier-assemblies 5.4  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [3]:
dataset = pt.get_dataset("vaswani")
index_ref = dataset.get_index()

In [4]:
type(index_ref)

jnius.reflect.org.terrier.querying.IndexRef

In [5]:
index = pt.IndexFactory.of(index_ref)
type(index)

jnius.reflect.org.terrier.structures.Index

In [6]:
print(index.getCollectionStatistics().toString())

Number of documents: 11429
Number of terms: 7756
Number of postings: 224573
Number of fields: 0
Number of tokens: 271581
Field names: []
Positions:   false



In [7]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
rm3 = pt.rewrite.RM3(index)

rm3_pipe = bm25 >> rm3 >> bm25

In [8]:
queries = dataset.get_topics()
queries.head()

,qid,query
0,1,measurement of dielectric constant of liquids ...
1,2,mathematical analysis and design details of wa...
2,3,use of digital computers in the design of band...
3,4,systems of data coding for information transfer
4,5,use of programs in engineering testing of comp...


In [9]:
bm25.transform(queries)

,qid,docid,docno,rank,score,query
0,1,8171,8172,0,24.566031,measurement of dielectric constant of liquids ...
1,1,9880,9881,1,22.110514,measurement of dielectric constant of liquids ...
2,1,5501,5502,2,21.717148,measurement of dielectric constant of liquids ...
3,1,1501,1502,3,19.478355,measurement of dielectric constant of liquids ...
4,1,9858,9859,4,18.626342,measurement of dielectric constant of liquids ...
...,...,...,...,...,...,...
91925,93,5495,5496,995,8.163870,high frequency oscillators using transistors t...
91926,93,9187,9188,996,8.163650,high frequency oscillators using transistors t...
91927,93,4821,4822,997,8.163588,high frequency oscillators using transistors t...
91928,93,9701,9702,998,8.163573,high frequency oscillators using transistors t...


In [11]:
queries_rm3 = (bm25 >> rm3).transform(queries)
queries_rm3

,qid,query_0,query
0,1,measurement of dielectric constant of liquids ...,applypipeline:off measur^0.100000009 breakdown...
1,2,mathematical analysis and design details of wa...,applypipeline:off detail^0.075000003 fed^0.075...
2,3,use of digital computers in the design of band...,applypipeline:off given^0.060000002 filter^0.1...
3,4,systems of data coding for information transfer,applypipeline:off code^0.260596812 transfer^0....
4,5,use of programs in engineering testing of comp...,applypipeline:off engin^0.167721525 test^0.150...
...,...,...,...
88,89,tunnel diode construction and its electrical c...,applypipeline:off multipli^0.028203228 convers...
89,90,electronic density of states at the surface of...,applypipeline:off adhes^0.028318578 depth^0.12...
90,91,resistivity of metallic thin films related to ...,applypipeline:off rough^0.100000009 comparison...
91,92,the phenomenon of radiation caused by charged ...,applypipeline:off excit^0.021660220 motion^0.0...


In [13]:
print(queries_rm3.iloc[0, 1])
print(queries_rm3.iloc[0, 2])
# NOTE see code https://github.com/terrier-org/pyterrier/blob/65e85dfa3d734e468d9eb5e5020423540d0e9efd/pyterrier/rewrite.py#L206
# applypipeline:off stops Terrier from re-stemming

measurement of dielectric constant of liquids by the use of microwave techniques
applypipeline:off measur^0.100000009 breakdown^0.032861188 materi^0.032861188 solut^0.030594902 liquid^0.149291784 solid^0.049291782 techniqu^0.100000009 dielectr^0.163456082 influenc^0.032861188 microwav^0.143059492 constant^0.100000009 strength^0.032861188 cathod^0.032861188


In [15]:
# TODO where is R from bm25 stored in (bm25 >> rm3)? how is it passed to folloring bm25 in rm3_pipe?

In [16]:
rm3_pipe.transform(queries)

,qid,docid,docno,rank,score,query
0,1,9880,9881,0,31.516689,applypipeline:off measur^0.100000009 breakdown...
1,1,8171,8172,1,22.195034,applypipeline:off measur^0.100000009 breakdown...
2,1,5501,5502,2,19.597914,applypipeline:off measur^0.100000009 breakdown...
3,1,1501,1502,3,18.615343,applypipeline:off measur^0.100000009 breakdown...
4,1,7922,7923,4,17.676662,applypipeline:off measur^0.100000009 breakdown...
...,...,...,...,...,...,...
92995,93,7261,7262,995,5.440783,applypipeline:off brief^0.019548872 switch^0.0...
92996,93,9405,9406,996,5.438101,applypipeline:off brief^0.019548872 switch^0.0...
92997,93,4951,4952,997,5.435846,applypipeline:off brief^0.019548872 switch^0.0...
92998,93,10483,10484,998,5.433339,applypipeline:off brief^0.019548872 switch^0.0...


In [17]:
pt.Experiment([bm25, rm3_pipe],
              queries,
              dataset.get_qrels(),
              ["map"]
              )

,name,map
0,BR(BM25),0.296517
1,"Compose(Compose(BR(BM25), <pyterrier.rewrite.R...",0.293326
